In [1]:
import dataiku
import json
#print(json.dumps(job,indent=4))

client = dataiku.api_client()
current_project = client.get_project(dataiku.get_custom_variables()["projectKey"])
# variables = current_project.get_variables()
# general_settings = client.get_general_settings()

In [2]:
ds_name_desired = 'SFPD_Reports_by_day'
# ds_name_default = 'SFPD_Reports_partitioned_PdDistrict_copy_NP'

dataset_desired = current_project.get_dataset(ds_name_desired)
# dataset_default = current_project.get_dataset(ds_name_default)

# dataiku.ComputedMetrics.get_global_data()
checks = dataset_desired.get_last_metric_values()
checks.get_all_ids()


[u'check:CHECK:min_1_column',
 u'basic:COUNT_COLUMNS',
 u'col_stats:COUNT:Descript',
 u'col_stats:COUNT:IncidntNum',
 u'col_stats:COUNT_DISTINCT:Descript',
 u'col_stats:COUNT_DISTINCT:IncidntNum',
 u'reporting:METRICS_COMPUTATION_DURATION',
 u'partitioning:COUNT_PARTITIONS',
 u'partitioning:PARTITIONS_LIST',
 u'records:COUNT_RECORDS',
 u'sql_query:metric_1:SQL probe',
 u'sql_query:metric_2:SQL probe']

In [12]:
# checks.get_metric_by_id(u'sql_query:metric_1:SQL probe')
print('one partition data: %s' %checks.get_partition_data(metric_id='basic:COUNT_COLUMNS',partition='2012-02-01'))
print('ALL partitions data: %s' %checks.get_partition_data(metric_id='basic:COUNT_COLUMNS',partition='ALL'))


one partition data: None
ALL partitions data: {u'dataType': u'BIGINT', u'partition': u'ALL', u'computed': 1645461013173, u'value': u'7'}


In [0]:
# #not using metrics fow now
# metrs.get_global_value('records:COUNT_RECORDS')
# partitions = dataset.list_partitions()
# partitions[0:3]

In [0]:
#compute_partitions=['2012-03-08','2012-03-13']
columns_of_interest=['Descript','IncidntNum']

In [0]:
#JSON for desired output
ds_settings_desired = dataset_desired.get_settings()

ds_metrics = ds_settings_desired.get_raw()['metrics']
ds_metrics_checks = ds_settings_desired.get_raw()['metricsChecks']

dataset_desired.compute_metrics(metric_ids=['basic:COUNT_COLUMNS'])

In [0]:
#JSON for desired output
# print(json.dumps(ds_metrics_desired['displayedState']['metrics'],indent=4))
# ds_metrics_desired['displayedState']['metrics']
# ['displayedState']['metrics']
#['probes'][3] pass full dict and parse ['configuration']['aggregates']


In [0]:
#JSON for default output
ds_settings_default = dataset_default.get_settings()
ds_metrics_default = ds_settings_default.get_raw()['metrics']

In [0]:
#JSON for default output
# print(json.dumps(ds_metrics_default['probes'],indent=4))
# ds_metrics_default['probes']
print(json.dumps(ds_metrics_default['displayedState']['metrics']))

In [0]:
# enable and define "Columns statistics" metrics
col_stat_json={u'computeOnBuildMode': u'NO',
 u'configuration': {u'aggregates': []},
 u'enabled': True,
 u'meta': {u'level': 2, u'name': u'Columns statistics'},
 u'type': u'col_stats'}

col_stat_aggr = col_stat_json['configuration']['aggregates']
# col_stat_aggr
col_stat_aggr.extend({u'aggregated': u'COUNT', u'column': cl} for cl in columns_of_interest)
col_stat_aggr.extend({u'aggregated': u'COUNT_DISTINCT', u'column': cl} for cl in columns_of_interest)
col_stat_aggr

col_stat_json['configuration']['aggregates'] = col_stat_aggr
col_stat_json
# type(col_stat_aggr)

In [0]:
# Status/Edit/General settings tab 
#in case any additional metrics are enabled or "Columns statistics" metrics were enabled but disabled afterwards
try:
    ds_metrics_default['probes'][3] = col_stat_json
except:
    ds_metrics_default['probes'].append(col_stat_json)

In [0]:
#Status/Metrics/Metrics to display pop-up
ds_metrics_default['displayedState']['metrics'].extend(compute_metrics)

In [0]:
ds_settings_default.save()

In [0]:
compute_metrics=[]
compute_metrics.extend('col_stats:COUNT:'+cl for cl in columns_of_interest)
compute_metrics.extend('col_stats:COUNT_DISTINCT:'+cl for cl in columns_of_interest)
compute_metrics

# compute_metrics=['col_stats:COUNT:Descript','col_stats:COUNT_DISTINCT:Descript','col_stats:COUNT:IncidntNum','col_stats:COUNT_DISTINCT:IncidntNum']

# dataset.compute_metrics(metric_ids=compute_metrics)

In [0]:
dataset_desired.compute_metrics(metric_ids=['basic:COUNT_COLUMNS','check:CHECK:min_1_column'])

In [0]:
# dataset_desired.run_checks(partition=["2012-03-31"],checks=["min_1_column"])
# Start call: /api/datasets/get-prepared-check-histories [POST] user=admin 
# [projectKey=PARTITIONED datasetName=SFPD_Reports_by_day data={"partition":"ALL","checks":["min_1_column"]}]
dataset_desired.run_checks()
